In [1]:
from datetime import datetime

from bs4 import BeautifulSoup
import pandas as pd
import requests

url = 'https://www.pro-football-reference.com/play-index/pgl_finder.cgi?request=1&match=game&year_min=2019&year_max=2019&season_start=1&season_end=-1&pos%5B%5D=QB&pos%5B%5D=WR&pos%5B%5D=RB&pos%5B%5D=TE&is_starter=E&game_type=R&career_game_num_min=1&career_game_num_max=400&qb_start_num_min=1&qb_start_num_max=400&game_num_min=0&game_num_max=99&week_num_min=0&week_num_max=99&c5val=1.0&order_by=fantasy_points&offset=0'

In [2]:
page = requests.get(url)
if page.status_code == 200:
    soup = BeautifulSoup(page.content, "lxml")
    table = soup.find('tbody')
    rows = table.findAll('tr')

In [3]:
col_list = ['log_id','player_id','player','pos','age','game_date','league_id',
    'team','game_location','opp','game_result','game_num','week_num',
    'game_day_of_week','fantasy_points','fantasy_points_ppr','draftkings_points','fanduel_points','pass_cmp','pass_att','pass_yds',
    'pass_td','pass_int','rush_att','rush_yds','rush_td','rec','rec_yds',
    'rec_td','fumbles','fgm','fga','xpm','xpa',]

In [4]:
df = pd.DataFrame(columns=col_list)

In [13]:
count = 0
for idx, item in enumerate(rows):
    data_set = rows[idx].findAll('td')
    for idx, item in enumerate(data_set):
        if idx == 0:
            df.loc[count,'player_id'] = item['data-append-csv']
            df.loc[count,'player'] = item.text
        else:
            if item['data-stat'] in col_list:
                try:
                    df.loc[count,item['data-stat']] = (datetime.strptime(item.text, '%Y-%m-%d') - datetime(1970, 1, 1)).total_seconds()
                except ValueError:
                    try:
                        df.loc[count,item['data-stat']] = round(float(item.text),2)
                    except ValueError:
                        df.loc[count,item['data-stat']] = item.text
    count += 1

In [33]:
df.log_id = df.player_id + '-' + df.game_date.astype(int).astype(str) + '-' + df.week_num.astype(int).astype(str)

In [34]:
df.to_csv('test.csv')